In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt

from datetime import date, datetime, timedelta

In [ ]:
from mainapp.views.dashboard import query_measurements_without_gaps

In [ ]:
from asgiref.sync import sync_to_async
from mainapp.tasks.metric_analyse import LOOKBACK_DAYS

@sync_to_async
def query(metric_id):
    end_date = date.today()
    start_date = end_date - timedelta(days=LOOKBACK_DAYS)
    results = query_measurements_without_gaps(start_date, end_date, metric_id)
    df = pd.DataFrame([{'date': m.date, 'value': m.value} for m in results]).set_index('date')
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
from mainapp.tasks.metric_analyse import STD_MULTIPLIER, TREND_ROLLING_DAYS, detected_spike
from mainapp.models import Metric

metric = await sync_to_async(Metric.objects.get)(name='XXX')
metric_id = metric.id
df = await query(metric_id)

df['trend'] = df['value'].rolling(TREND_ROLLING_DAYS).mean()
std = df['trend'].std()
ax = df[['value', 'trend']].plot(style='.-', title=metric.name);
# add noise level
d = df.index.to_pydatetime()
plt.fill_between(d, df['trend'] - std * STD_MULTIPLIER, df['trend'] + std * STD_MULTIPLIER, alpha=0.2)
# detect points
df['is_spike'] = (~df['value'].isna()) & ((df['trend'] - df['value']).abs() > std * STD_MULTIPLIER)
# mark
df['value'].where(df['is_spike']).plot(style='o', color='red')

# Check current implementation
await sync_to_async(detected_spike)(metric_id)